# PyTrilinos2 tutorial

We will use the Python interface of Trilinos (PyTrilinos2) to set up and solve a linear system (using Tpetra and Thyra) using different linear solvers and preconditioners from Amesos2, Belos, Ifpack2 and MueLu.

We first set up a MPI communicator and an output stream. The MPI communicator has size 1 for now.

In [ ]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
# get C-level cout to display
%load_ext wurlitzer

In [ ]:
from mpi4py import MPI
from PyTrilinos2 import Teuchos

comm = Teuchos.getTeuchosComm(MPI.COMM_WORLD)
out = Teuchos.fancyOStream(Teuchos.getCout())
print("Comm size", comm.getSize())

The Tpetra package provides sparse linear algebra.

In [ ]:
from PyTrilinos2 import Tpetra

Tpetra linear algebra classes are templated on
- scalar type, e.g. `double`, `float`, `std::complex<double>`
- local ordinal type, normally `int`
- global ordinal type, normally `long long`
- node type, a combination of a Kokkos execution space and memory space.
  
Python does not have templates, so templated classes are exposed with template parameters as suffixes:

In [ ]:
Tpetra.Map_int_long_long_Tpetra_KokkosCompat_KokkosDeviceWrapperNode_Kokkos_Serial_t

Since that's not very convenient (and differs from C++) we have some helper classes:

In [ ]:
Tpetra.Map['int', 'long long', 'serial']

We can leverage default values for template arguments to further simplify:

In [ ]:
Tpetra.Map['int']

In [ ]:
Tpetra.Map[()]  # no arguments specified, note ()

We set up a linear system
$$A x = b$$

In [ ]:
def assemble1DLaplacian(rowmap):
    from PyTrilinos2 import Tpetra

    A = Tpetra.CrsMatrix[()](rowmap, 3)
    for lid in range(rowmap.getMinLocalIndex(), rowmap.getMaxLocalIndex()+1):
        gid = rowmap.getGlobalElement(lid)
        indices = [gid]
        vals = [2.]
        if gid > 0:
            indices.append(gid-1)
            vals.append(-1.)
        if gid < rowmap.getMaxAllGlobalIndex():
            indices.append(gid+1)
            vals.append(-1.)
        A.insertGlobalValues(gid, indices, vals)
    A.fillComplete()
    return A

numRows = 1000
tpetra_map = Tpetra.Map[()](numGlobalElements=numRows, numLocalElements=numRows, indexBase=0, comm=comm)    
tpetra_A = assemble1DLaplacian(tpetra_map)

tpetra_b = Tpetra.Vector['double'](tpetra_A.getRangeMap())
tpetra_x = Tpetra.Vector['double'](tpetra_A.getDomainMap())
tpetra_b.putScalar(1.0)
tpetra_x.putScalar(0.0)

# compute the residual and check that x does not solve the system already
tpetra_residual = Tpetra.Vector['double'](tpetra_A.getRangeMap())
tpetra_A.apply(tpetra_x, tpetra_residual)
tpetra_residual.update(1.0, tpetra_b, -1.0)
print(r'|Ax-b|_2 =', tpetra_residual.norm2())

We want to solve for $x$. 
Trilinos provides
- direct solvers in the package Amesos2,
- Krylov solvers in Belos,
- simple preconditioners and incomplete factorizations in Ifpack2,
- algebraic multigrid in MueLu,
- and more.

Trilinos has a unified solver/preconditioner interface (Stratimikos).
We will wrap our linear algebra objects from Tpetra to Thyra so that Stratimikos can use them.

In [ ]:
from PyTrilinos2 import Thyra


def toThyra(obj):
    if isinstance(obj, Tpetra.Map.templatedClasses):
        return Thyra.tpetraVectorSpace(obj)
    elif isinstance(obj, Tpetra.Vector.templatedClasses):
        thyra_map = toThyra(obj.getMap())
        return Thyra.tpetraVector(thyra_map, obj)
    elif isinstance(obj, Tpetra.MultiVector.templatedClasses):
        thyra_map = toThyra(obj.getMap())
        return Thyra.tpetraMultiVector(thyra_map, obj)
    elif isinstance(obj, Tpetra.CrsMatrix.templatedClasses):
        thyra_domainmap = toThyra(obj.getDomainMap())
        thyra_rangemap = toThyra(obj.getRangeMap())
        return Thyra.tpetraLinearOp(thyra_rangemap, thyra_domainmap, obj)
    else:
        raise NotImplementedError(type(obj))

thyra_x = toThyra(tpetra_x)
thyra_b = toThyra(tpetra_b)
thyra_A = toThyra(tpetra_A)

Now we can set up the linear solver builder:

In [ ]:
from PyTrilinos2 import Stratimikos

linearSolverBuilder = Stratimikos.LinearSolverBuilder['double']()

We set parameters as a Teuchos ParameterList which is similar to a Python dictionary. "Amesos2" will give us a direct solver.

In [ ]:
params = Teuchos.ParameterList()
params["Linear Solver Type"] = "Amesos2"
print(params)

In [ ]:
linearSolverBuilder.setParameterList(params)
solverFactory = Thyra.createLinearSolveStrategy(linearSolverBuilder)
thyra_invA = Thyra.linearOpWithSolve(solverFactory, thyra_A)

We can now solve the linear system:

In [ ]:
status = thyra_invA.solve(Thyra.NOTRANS, thyra_b, thyra_x)
status.solveStatus

We check the result:

In [ ]:
tpetra_A.apply(tpetra_x, tpetra_residual)
tpetra_residual.update(1.0, tpetra_b, -1.0)
print(r'|Ax-b|_2 =', tpetra_residual.norm2())

Print default parameters

In [ ]:
print(linearSolverBuilder.getValidParameters())

In [ ]:
Stratimikos.enableMueLu(linearSolverBuilder, "MueLu")

In [ ]:
params = Teuchos.ParameterList()
params["Preconditioner Type"] = "MueLu"
linearSolverBuilder.setParameterList(params)
solverName = linearSolverBuilder.getLinearSolveStrategyName()
precName = linearSolverBuilder.getPreconditionerStrategyName()
solverFactory = Thyra.createLinearSolveStrategy(linearSolverBuilder)
solverName, precName

In [ ]:
thyra_invA = Thyra.linearOpWithSolve(solverFactory, thyra_A)

In [ ]:
status = thyra_invA.solve(Thyra.NOTRANS, thyra_b, thyra_x)
status.message

In [ ]:
tpetra_b.getLocalViewHost()

In [ ]:
tpetra_x.getLocalViewHost()

In [ ]:
plt.plot(tpetra_x.getLocalViewHost())